In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

In [3]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
# !pip install transformers==4.36.2

In [21]:
from datasets import load_dataset

dataset_name = 'clarin-knext/summarization-chat-annotated'
dataset = load_dataset(dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'doc_text', 'summary'],
        num_rows: 32480
    })
})

In [22]:
dataset['train'][0]

{'id': 'complexqa_806054',
 'doc_text': 'Zamach w Sarajewie Zamach w Sarajewie – zamach na następcę austro-węgierskiego tronu, arcyksięcia Franciszka Ferdynanda i jego żonę Zofię, księżnę Hohenberg, dokonany 28 czerwca 1914 roku przez bośniackiego Serba Gavrila Principa, członka serbskiej nacjonalistycznej organizacji "Młoda Bośnia". Princip wraz z pięcioma innymi uczestnikami zamachu powiązany był z serbską tajną organizacją "Zjednoczenie lub śmierć", popularnie nazywaną "Czarna Ręka", którą kierował Dragutin Dimitrijević pseudonim "Apis", szef serbskiego wywiadu wojskowego. Morderstwo następcy tronu austro-węgierskiego stworzyło napięcie pomiędzy Wiedniem a Belgradem. Austro-Węgry wysunęły ultimatum domagając się usunięcia wrogiej propagandy, a także udziału reprezentantów rządu cesarsko-królewskiego w śledztwie na terenie Serbii. Odrzucenie części żądań przez Serbię doprowadziło do tzw. kryzysu lipcowego, a w konsekwencji do wybuchu I wojny światowej. Przygotowania Przygotowania do 

In [13]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# model_name = "speakleash/Bielik-7B-v0.1"
model_name = "speakleash/Bielik-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_8bit=True, device_map="auto")


/home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.66s/it]
/home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
doc_text = dataset['train'][0]['doc_text']

def get_prompt(doc_text):
    return f"Twoim zadaniem jest przeczytanie podanego tekstu i napisanie streszczenia w języku polskim. Streszczenie powinno zawierać najważniejsze informacje i wydarzenia opisane w tekście, być zwięzłe i dobrze zorganizowane. Unikaj wprowadzania nowych informacji oraz osobistych opinii.\n\n###\n\n{doc_text}\n"

print(get_prompt(doc_text))

Twoim zadaniem jest przeczytanie podanego tekstu i napisanie streszczenia w języku polskim. Streszczenie powinno zawierać najważniejsze informacje i wydarzenia opisane w tekście, być zwięzłe i dobrze zorganizowane. Unikaj wprowadzania nowych informacji oraz osobistych opinii.

###

Zamach w Sarajewie Zamach w Sarajewie – zamach na następcę austro-węgierskiego tronu, arcyksięcia Franciszka Ferdynanda i jego żonę Zofię, księżnę Hohenberg, dokonany 28 czerwca 1914 roku przez bośniackiego Serba Gavrila Principa, członka serbskiej nacjonalistycznej organizacji "Młoda Bośnia". Princip wraz z pięcioma innymi uczestnikami zamachu powiązany był z serbską tajną organizacją "Zjednoczenie lub śmierć", popularnie nazywaną "Czarna Ręka", którą kierował Dragutin Dimitrijević pseudonim "Apis", szef serbskiego wywiadu wojskowego. Morderstwo następcy tronu austro-węgierskiego stworzyło napięcie pomiędzy Wiedniem a Belgradem. Austro-Węgry wysunęły ultimatum domagając się usunięcia wrogiej propagandy, a t

In [15]:
# !pip install alpaca-eval

In [25]:
inputs['input_ids'].shape

torch.Size([1, 2078])

In [34]:
alpaca_eval = load_dataset(
    "tatsu-lab/alpaca_eval", "alpaca_eval"
)["eval"]

In [36]:
alpaca_eval

Dataset({
    features: ['instruction', 'output', 'generator', 'dataset'],
    num_rows: 805
})

In [35]:
alpaca_eval[0]

{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
 'generator': 'text_davinci_003',
 'dataset': 'helpful_base'}

In [ ]:
alpaca_eval

In [69]:
import datasets
from tqdm import tqdm

eval_data = []
N = 20
temp = 0.2
top_p = 0.4

for i, example in tqdm(enumerate(dataset["train"])):
    if i == N:
        break
    prompt = get_prompt(example["doc_text"])
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    inputs_length = len(inputs['input_ids'][0])
    output = model.generate(**inputs, temperature=temp, top_p=top_p, max_new_tokens=int(inputs_length * 0.1))
    generated_text = tokenizer.decode(output[0][inputs_length:], skip_special_tokens=True)
    eval_data.append({"instruction": prompt, "output": generated_text, "generator": model_name.split("/")[-1], "dataset": dataset_name.split("/")[-1], "datasplit": "train"})

0it [00:00, ?it/s]/home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
1it [00:50, 50.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2it [01:03, 28.29s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
3it [01:52, 37.65s/it]Setting `pad_token_id` to `eos_token_id`:2 for

In [ ]:
len(eval_data)

10

In [ ]:
from copy import deepcopy

reference_data = deepcopy(eval_data)

for i, example in tqdm(enumerate(dataset["train"])):
    if i == N:
        break
    reference_data[i]["output"] = example["summary"]



10it [00:00, 5279.17it/s]


In [ ]:
reference_data

[{'instruction': 'Twoim zadaniem jest przeczytanie podanego tekstu i napisanie streszczenia w języku polskim. Streszczenie powinno zawierać najważniejsze informacje i wydarzenia opisane w tekście, być zwięzłe i dobrze zorganizowane. Unikaj wprowadzania nowych informacji oraz osobistych opinii.\n\n###\n\nZamach w Sarajewie Zamach w Sarajewie – zamach na następcę austro-węgierskiego tronu, arcyksięcia Franciszka Ferdynanda i jego żonę Zofię, księżnę Hohenberg, dokonany 28 czerwca 1914 roku przez bośniackiego Serba Gavrila Principa, członka serbskiej nacjonalistycznej organizacji "Młoda Bośnia". Princip wraz z pięcioma innymi uczestnikami zamachu powiązany był z serbską tajną organizacją "Zjednoczenie lub śmierć", popularnie nazywaną "Czarna Ręka", którą kierował Dragutin Dimitrijević pseudonim "Apis", szef serbskiego wywiadu wojskowego. Morderstwo następcy tronu austro-węgierskiego stworzyło napięcie pomiędzy Wiedniem a Belgradem. Austro-Węgry wysunęły ultimatum domagając się usunięcia w

In [ ]:
import json

with open("eval_data.json", "w") as f:
    json.dump(eval_data, f)


with open("reference_data.json", "w") as f:
    json.dump(reference_data, f)

In [55]:
!export OPENAI_API_KEY=...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!alpaca_eval --model_outputs 'eval_data.json' --reference_outputs 'reference_data.json'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:root:Evaluating the Bielik-7B-Instruct-v0.1 outputs.
INFO:root:Creating the annotator from `weighted_alpaca_eval_gpt4_turbo`.
INFO:root:Saving annotations to `/home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/weighted_alpaca_eval_gpt4_turbo/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/weighted_alpaca_eval_gpt4_turbo/annotations_seed0_configs.json.
Annotation chunk:   0%|                                   | 0/1 [00:00<?, ?it/s]INFO:root:Annotating 0 examples with weighted_alpaca_eval_gpt4_turbo
INFO:root:Saving all annotations to /home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/weighted_alpaca_eval_gpt4_turbo/annotations_seed0_configs.json.
INFO:root:Loading all annotations from /home/djaniak/anaconda3/envs/rlhf-gen-div/lib/python3.10/site-packages/alp

In [7]:
# !pip install bitsandbytes

In [14]:
from tqdm import tqdm

outputs = []


prompt_input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

summarization_ratio_lenth = 0.15
max_length = int(len(prompt_input_ids[0]) * (1 + summarization_ratio_lenth))
print(max_length)

for _ in tqdm(range(16)):
    with torch.no_grad():
        output = model.generate(prompt_input_ids.to(device),
                                pad_token_id=tokenizer.eos_token_id,
                                do_sample=True, 
                                max_length=max_length,
                                temperature=1.0, 
                                top_k=0,
                                top_p=1
                                )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(generated_text)
    print("==============================")

2556


  6%|▋         | 1/16 [01:10<17:30, 70.03s/it]

Twoim zadaniem jest przeczytanie podanego tekstu i napisanie streszczenia w języku polskim. Streszczenie powinno zawierać najważniejsze informacje i wydarzenia opisane w tekście, być zwięzłe i dobrze zorganizowane. Unikaj wprowadzania nowych informacji oraz osobistych opinii.

###

Tekst: Zamach w Sarajewie Zamach w Sarajewie – zamach na następcę austro-węgierskiego tronu, arcyksięcia Franciszka Ferdynanda i jego żonę Zofię, księżnę Hohenberg, dokonany 28 czerwca 1914 roku przez bośniackiego Serba Gavrila Principa, członka serbskiej nacjonalistycznej organizacji "Młoda Bośnia". Princip wraz z pięcioma innymi uczestnikami zamachu powiązany był z serbską tajną organizacją "Zjednoczenie lub śmierć", popularnie nazywaną "Czarna Ręka", którą kierował Dragutin Dimitrijević pseudonim "Apis", szef serbskiego wywiadu wojskowego. Morderstwo następcy tronu austro-węgierskiego stworzyło napięcie pomiędzy Wiedniem a Belgradem. Austro-Węgry wysunęły ultimatum domagając się usunięcia wrogiej propagan

  6%|▋         | 1/16 [01:16<19:06, 76.46s/it]


KeyboardInterrupt: 

In [50]:
len(output[0])

2503

In [51]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Twoim zadaniem jest przeczytanie podanego tekstu i napisanie streszczenia w języku polskim. Streszczenie powinno zawierać najważniejsze informacje i wydarzenia opisane w tekście, być zwięzłe i dobrze zorganizowane. Unikaj wprowadzania nowych informacji oraz osobistych opinii.\n\n###\n\nTekst: Zamach w Sarajewie Zamach w Sarajewie – zamach na następcę austro-węgierskiego tronu, arcyksięcia Franciszka Ferdynanda i jego żonę Zofię, księżnę Hohenberg, dokonany 28 czerwca 1914 roku przez bośniackiego Serba Gavrila Principa, członka serbskiej nacjonalistycznej organizacji "Młoda Bośnia". Princip wraz z pięcioma innymi uczestnikami zamachu powiązany był z serbską tajną organizacją "Zjednoczenie lub śmierć", popularnie nazywaną "Czarna Ręka", którą kierował Dragutin Dimitrijević pseudonim "Apis", szef serbskiego wywiadu wojskowego. Morderstwo następcy tronu austro-węgierskiego stworzyło napięcie pomiędzy Wiedniem a Belgradem. Austro-Węgry wysunęły ultimatum domagając się usunięcia wrogiej pro

In [53]:
tokenizer.decode(output[0][len(input_ids[0]):], skip_special_tokens=True).strip()

'Praca dotyczy zamachu an ulicach Sarajewa sprzed 100 lat. Odniesionw wtedy rany zmarłego 1 czerwca 1914 roku Arcyksięcia Franciszka Ferdynanda doprowadziły do wybuchu I Wojny światowej. Autorka opisuje w pracy kulisy przygotowania oraz sam przebbieg ataku zamachowca. Podkreślając pierwsze kroki spiskowców jak: wizę w konsulacie austriackiego w Berlinie, misje wywiadowcze w Belgradzie, gromadzenie informacji na temat pary arcyksiążęcej zabijanie niewinnych osób dla zyskania ważnych informacji w formie zmylni przed śmiercią. Ciekawe kolejne strzały, treść "brat by wolał a ty wolisz brata" oraz niewłaściwe zachowywanie zamachowców, którzy musieli postrzelić ofiary żeby później je unieszkodliwić.'

In [16]:
#read json
import json

with open('../Bielik-7B-v0.1_summarization-chat-annotated.json') as f:
    data = json.load(f)

In [20]:
data[0]

['1. 2. 3. 4. 5. podsumowanie',
 'Zamach w Sarajewie był spowodowany przez Gavrila Principa i sześciu innych członków Zjednoczenia lub śmierci. Cel zamachu był arcyksiążę Franciszek Ferdynand. Zamach miał miejsce niedaleko Bałkanów 28 czerwca 1914 roku. Opisany incydent spowodował śmierć dwóch osób. Po tym armia austro-węgierska wypowiedziała wojnę Niemcom, co rozpoczęło I wojnę światową.\n\nPrzypisy \n\nzamach w Sarajewie\nzamach w Sarajewie, Streszczenie',
 'Artykuł opowiada o następcy tronu Austro-Węgier Francisz Kawka, który zginął w zamachu 28 czerwca 1914 roku w Sarajewie. Przeprowadzony został zamach przez serbskich bojowników sprowokowany przez rząd austracko-węgierski. Zamach obalił porozumienie z Serbinją i doprowadził do I wojny światowej. Princip, jeden z uczestników zamachu, został opisany jako zamachowiec, który próbował trzykrotnie zabić Francisa.',
 'Latem 1914 roku miało miejsce wydarzenie zwane czarnym zamachem. Zamach ten został przeprowadzony przez Serbów. Według mn

In [ ]:
from tqdm import tqdm

N = 16
temperature = 1.0  
outputs = []

for example in tqdm(dataset["train"].select(range(1000))):
    input_ids = tokenizer.encode(example["doc_text"], return_tensors="pt")
    samples = []
    for _ in range(N):
        output = model.generate(input_ids.to(device),
                                pad_token_id=tokenizer.eos_token_id,
                                do_sample=True, 
                                max_length=50, 
                                temperature=temperature, 
                                top_k=0,
                                top_p=1
                                )
        
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        samples.append(generated_text)
    outputs.append(samples)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/djaniak/mambaforge/envs/rlhf-gen-div-new/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 2102, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 1/1000 [00:23<6:33:29, 23.63s/it]/home/djaniak/mambaforge/envs/rlhf-gen-div-new/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 419, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 2/1000 [00:27<3:21:01, 12.09s/it]/home/djaniak/mambaforge/envs/rlhf-gen-div-new/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 2048, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `m

KeyboardInterrupt: 

In [10]:
model_name = "speakleash/Bielik-Instruct-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

NameError: name 'AutoTokenizer' is not defined